<a href="https://colab.research.google.com/github/smnieee/ml_workshop/blob/master/session5/1_FunWithNeuralNets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fun with Neural Networks

---

## Southern Minnesota IEEE: Machine Learning Workshop 5
## May 16, 2022

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers

## Simple Perceptron Example (Classification)

The perceptron is the basic unit of the neural network. It takes multiple inputs and produces one output. The inputs are adjusted with weights, $w$, and biases, $b$. The output is a result of an activation function being applied to the sum of the adjusted inputs.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

In [ ]:
# Load data and view them
iris = load_iris()

iris

In [ ]:
# Let's look at petal length and width
X = iris.data[:, 2:4]

# Plot the data
fig, ax = plt.subplots()
for lbl,name in enumerate(iris.target_names):
  ax.scatter(X[(iris.target==lbl),0], X[(iris.target==lbl),1],
             label=name)

ax.legend(title='Species')
ax.set_xlabel('Petal Length')
ax.set_ylabel('Petal Width')
plt.show()

In [ ]:
# Binary targets: it's either Setosa or not
y = (iris.target == 0).astype(int)

In [ ]:
# Create a Perceptron from Scikit-Learn and use it to predict setosa
is_setosa = Perceptron()
is_setosa.fit(X,y)

y_pred = is_setosa.predict([[3.,1.]])
y_pred.astype(bool)

## Linear Regression Example

The petal length versus width appears to be linear. This can also be approximated with a simple network.

In [ ]:
from sklearn import linear_model
petal_regression = linear_model.LinearRegression()

x_linreg = X[:,0].reshape(-1,1)
y_linreg = X[:,1].reshape(-1,1)

petal_regression.fit(x_linreg,y_linreg)

new_X = np.linspace(1,7).reshape(-1,1)
new_Y = petal_regression.predict(new_X)

print("Coefficient: ", petal_regression.coef_)
print("Intercept: ", petal_regression.intercept_)

# Plot the data
fig, ax = plt.subplots()
for lbl,name in enumerate(iris.target_names):
  ax.scatter(X[(iris.target==lbl),0], X[(iris.target==lbl),1],
             label=name)

ax.legend(title='Species')
ax.set_xlabel('Petal Length')
ax.set_ylabel('Petal Width')

ax.plot(new_X, new_Y)
plt.show()


## Full Classification Example (Keras)

A neural network comprises many artificial neurons like the perceptron. The neurons are orgainized by layer. The outputs from one layer are appropriately modified and passed as inputs to the next layer. Each layer is optimized using techniques such as gradient descent.



In [ ]:
import sklearn
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Rescale the data
X_train_full = X_train_full /255.0 
X_test = X_test / 255.0 

print("Train Image Array Shape: ", np.shape(X_train_full))
print("Train Label Array Shape: ", np.shape(y_train_full))

#---
# View the Test Data Here
#---

img_samples = np.random.randint(0, high=len(X_train_full), size=9)

fig,ax = plt.subplots(3,3)

for n in range(9):
  ind = np.unravel_index(n, (3,3))
  img = X_train_full[img_samples[n]]
  ax[ind].imshow(img)

plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

TRAIN_SPLIT = 0.66
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full,
                                                  train_size=TRAIN_SPLIT)

# CHECK the size of the arrays like above


### Build the Keras Model

The Sequential model is a common model to start with. It assumes a linear sequence of layers like a pipline.

In [ ]:
# Create an instance of the model to build on
model = keras.models.Sequential([
  # Add the input layer. In this scenario we will just flatten the image array
  keras.layers.Flatten(input_shape=[28,28]),
  # Change the size of the first layer
  keras.layers.Dense(100, activation='relu'),
  #Try More layers
  keras.layers.Dense(10, activation='softmax')
])

### Review the model

It is important to look at your model after building it. Some useful functions are `model.summary()` and `model.layers`.

In [ ]:
model.summary()

In [ ]:
for l in model.layers:
  print(l.name, " weights: ", l.get_weights())

### Compile the Model

In Keras / Tensorflow, you compile the model before using it.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

### Train the Model

Next you need to train the model on our training data.

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_val, y_val))

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot()
plt.show()

### Evaluate the Model on Test Data

In [ ]:
model.evaluate(X_test, y_test)

---

## References

Géron, Aurélien. Hands-on machine learning with Scikit-Learn, Keras, and TensorFlow: Concepts, tools, and techniques to build intelligent systems. " O'Reilly Media, Inc.", 2019.

